In [2]:
import io
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold, cross_val_predict, train_test_split
import pylab
import seaborn as sns
sns.set(style="ticks", color_codes=True, font_scale=1.5)
from matplotlib import pyplot as plt
from IPython.display import Image
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.metrics import auc, accuracy_score

In [3]:
data_path = './bank-additional.csv'

df = pd.read_csv(data_path,sep=';').dropna()
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no


In [4]:
categorical_feature_mask = df.dtypes==object
categorical_cols = df.columns[categorical_feature_mask].tolist()
print(df[categorical_cols])

              job  marital          education default  housing     loan  \
0     blue-collar  married           basic.9y      no      yes       no   
1        services   single        high.school      no       no       no   
2        services  married        high.school      no      yes       no   
3        services  married           basic.9y      no  unknown  unknown   
4          admin.  married  university.degree      no      yes       no   
...           ...      ...                ...     ...      ...      ...   
4114       admin.  married           basic.6y      no      yes      yes   
4115       admin.  married        high.school      no      yes       no   
4116      student   single        high.school      no       no       no   
4117       admin.  married        high.school      no       no       no   
4118   management   single        high.school      no      yes       no   

        contact month day_of_week     poutcome   y  
0      cellular   may         fri  nonexistent

In [5]:
le = LabelEncoder()
df[categorical_cols] = df[categorical_cols].apply(le.fit_transform)
# df[categorical_cols] = df[categorical_cols].dropna(inplace=True)
df_new = df[categorical_cols].dropna()
df_new.head(10)

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
0,1,1,2,0,2,0,0,6,0,1,0
1,7,2,3,0,0,0,1,6,0,1,0
2,7,1,3,0,2,0,1,4,4,1,0
3,7,1,2,0,1,1,1,4,0,1,0
4,0,1,6,0,2,0,0,7,1,1,0
5,7,2,6,0,0,0,0,9,2,0,0
6,0,2,6,0,2,0,0,9,1,1,0
7,2,1,6,1,2,0,0,7,1,1,0
8,7,0,5,0,0,0,0,7,3,0,0
9,1,1,2,1,0,0,1,6,2,1,0


In [6]:
Y = df['y']
X = df.drop('y', axis=1)
scalers = StandardScaler()
X = scalers.fit_transform(X)

In [7]:
X

array([[-0.98075178, -0.7834306 , -0.29192536, ..., -1.240939  ,
        -1.33170721, -0.91477933],
       [-0.10799146,  0.88051782,  1.357513  , ...,  0.89226907,
         0.71169841,  0.3328625 ],
       [-1.46561862,  0.88051782, -0.29192536, ..., -0.28317211,
         0.77342749,  0.83653509],
       ...,
       [-1.27167189,  1.15784256,  1.357513  , ..., -1.240939  ,
        -1.30805401, -0.91477933],
       [ 1.73450254, -1.06075533, -0.29192536, ...,  0.95757136,
         0.77573511,  0.83653509],
       [-0.59285831,  0.04854361,  1.357513  , ..., -0.32670697,
         0.28767155,  0.39802768]])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    train_size=0.8,
                                                    test_size = 0.2, random_state=500)

In [9]:
model = Sequential()
model.add(Dense(40, input_shape=(20,)))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [10]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
history = model.fit(X_train, y_train,batch_size=50,epochs=50,validation_split=0.2, shuffle=True,verbose=1)
y_pred = model.predict(X_test)

Epoch 1/50
53/53 [==============================] - 0s 6ms/step - loss: 0.7321 - accuracy: 0.5046 - val_loss: 0.5106 - val_accuracy: 0.8225
Epoch 2/50
53/53 [==============================] - 0s 1ms/step - loss: 0.4096 - accuracy: 0.8832 - val_loss: 0.3128 - val_accuracy: 0.9302
Epoch 3/50
53/53 [==============================] - 0s 1ms/step - loss: 0.2991 - accuracy: 0.9048 - val_loss: 0.2394 - val_accuracy: 0.9302
Epoch 4/50
53/53 [==============================] - 0s 1ms/step - loss: 0.2550 - accuracy: 0.9055 - val_loss: 0.2075 - val_accuracy: 0.9317
Epoch 5/50
53/53 [==============================] - 0s 2ms/step - loss: 0.2347 - accuracy: 0.9040 - val_loss: 0.1907 - val_accuracy: 0.9378
Epoch 6/50
53/53 [==============================] - 0s 2ms/step - loss: 0.2226 - accuracy: 0.9040 - val_loss: 0.1829 - val_accuracy: 0.9363
Epoch 7/50
53/53 [==============================] - 0s 2ms/step - loss: 0.2143 - accuracy: 0.9055 - val_loss: 0.1784 - val_accuracy: 0.9347
Epoch 8/50
53/53 [==

In [14]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list)==0:
        print('Loss is missing in history')
    return loss_list, val_loss_list, acc_list, val_acc_list
        

In [15]:
loss_list, val_loss_list, acc_list, val_acc_list = plot_history(history)

In [17]:
print(loss_list)

['loss']
